In [1]:
import statsmodels.api as sm
import glob
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [2]:
#**IMPORTING DATA**
files=glob.glob(f'../../data/train/*_solo_linear_model.csv')

#getting the prices history from each index
data=[]
for file in files:
    data.append(pd.read_csv(file))

#converting the 'date' columns to datetime type
for i in data:
    i['date']=pd.to_datetime(i['date'])
    i.set_index('date',inplace=True)

In [3]:
df={}
for i in data:
    if 'close' in i.columns[0]:
        df[i.columns[0]]=i

In [4]:
tar=pd.DataFrame()

In [5]:
preds=pd.DataFrame()
for i,j in df.items():
    x=j.drop(columns=[i])
    x=sm.add_constant(x)
    y=j[i]
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.011,shuffle = False)
    tar[i]=y_test
    model= Ridge(fit_intercept=False)
    model.fit(X_train, y_train)
    preds[i]=model.predict(X_test)

In [6]:
preds=preds.set_index(y_test.index)

In [7]:
temp=[]
for i in preds.columns:
    temp.append(i+'_solo_linear_model')
preds.columns=temp
preds.index=y_test.index

In [8]:
preds

,ada_close_solo_linear_model,bit_close_solo_linear_model,bnb_close_solo_linear_model,eth_close_solo_linear_model
date,,,,
2021-08-31,2.880248,47625.666813,477.303165,3281.313974
2021-09-01,2.773592,47109.332246,452.380003,3402.880956
2021-09-02,2.898139,48245.556645,496.260987,3768.917951
2021-09-03,2.907049,49710.245619,472.212174,3823.920490
2021-09-07,3.053427,50078.184166,496.664456,3920.440204
2021-09-08,2.653468,47044.826958,418.352461,3479.200009
2021-09-09,2.538275,46538.170765,414.102807,3549.702641
2021-09-10,2.557730,46097.043008,423.108742,3402.111490
2021-09-13,2.470955,44886.402160,391.519870,3217.816938


In [9]:
tar

,ada_close,bit_close,bnb_close,eth_close
date,,,,
2021-08-31,2.772475,46919.511555,464.418732,3388.026327
2021-09-01,2.870767,48487.011189,490.068726,3771.524040
2021-09-02,2.963438,49725.532134,483.921051,3805.823968
2021-09-03,2.968239,49709.611994,488.604706,3879.477515
2021-09-07,2.502491,46897.601477,417.508789,3445.243837
2021-09-08,2.474989,46769.314391,415.113159,3559.717504
2021-09-09,2.517666,46404.364481,421.270020,3440.940392
2021-09-10,2.385911,44720.755550,401.621796,3205.832088
2021-09-13,2.400663,45097.797218,399.063446,3294.403758


In [10]:
targets=['bit_close','eth_close','ada_close','bnb_close']
for target in targets:
    temp_y=tar[target]
    y_hat=[i for i in preds.columns if target in i]
    y_hat=preds[y_hat]
    sse=0
    for i in range(len(temp_y)):
        sse+=(temp_y.values[i]-y_hat.values[i][0])**2
    aic=sse/7
    aic=7*np.log(aic)+2
    print(target,'AIC:',aic)

bit_close AIC: 106.87597732543581
eth_close AIC: 80.17061531158468
ada_close AIC: -17.713011244536553
bnb_close AIC: 52.91686092763398


In [11]:
preds.to_csv(f'../../data/predictions/solo_linear_model.csv')